In [ ]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=d617ded871af704edaddecbd55275353b44480e13ceb9a888256147bfb665d6d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import requests
import regex as re
import wget
import os

In [ ]:
from re import X
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import pandas as pd
from argparse import ArgumentParser, Namespace
from pathlib import Path
from typing import Dict
from datetime import datetime
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import *
import numpy as np
import xarray as xr

In [ ]:
sd = datetime.datetime(2021, 1, 1).strftime("%Y%m%d")
ed = datetime.datetime(2021, 12, 31).strftime("%Y%m%d")

In [ ]:
ls=pd.date_range(sd,ed)

In [ ]:
dates=[dt.strftime("%Y%m%d") for dt in ls]

In [ ]:
years={date[:4] for date in dates}
months={date[4:6] for date in dates}

In [ ]:
txt=""
for year in years:
    for month in months:
        base='https://www.ngdc.noaa.gov/dscovr/data/'+year+'/'+month+'/'
        txt+=requests.get(base).text

In [ ]:
urllist=[]

In [ ]:
for date in dates:
    base='https://www.ngdc.noaa.gov/dscovr/data/'+date[:4]+'/'+date[4:6]+'/'
    x=re.search(str("oe_m1m_dscovr_s"+date+"000000_"), txt).start()
    url=base+txt[x: x+71]
    print(date,": ",url)
    urllist.append(url[46:])
    wget.download(url)

20210101 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210101000000_e20210101235959_p20210102020312_pub.nc.gz
20210102 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210102000000_e20210102235959_p20210103020543_pub.nc.gz
20210103 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210103000000_e20210103235959_p20210104020454_pub.nc.gz
20210104 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210104000000_e20210104235959_p20210105020023_pub.nc.gz
20210105 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210105000000_e20210105235959_p20210106020421_pub.nc.gz
20210106 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210106000000_e20210106235959_p20210107020234_pub.nc.gz
20210107 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210107000000_e20210107235959_p20210108020412_pub.nc.gz
20210108 :  https://www.ngdc.noaa.gov/dscovr/data/2021/01/oe_m1m_dscovr_s20210108000000_e2

In [ ]:
l=len(urllist)

In [ ]:
magdata=[]

In [ ]:
for i in range(0,l):
  ds=xr.open_dataset('/content/'+urllist[i])
  df=ds.to_dataframe()
  actualdf=df.dropna()
  df_len=len(actualdf)
  for j in range(0,df_len,60):
   bx_gse_hour=np.nanmean(np.array(actualdf['bx_gse'][j:j+60]))
   by_gse_hour=np.nanmean(np.array(actualdf['by_gse'][j:j+60]))
   bz_gse_hour=np.nanmean(np.array(actualdf['bz_gse'][j:j+60]))
   magdata.append([bx_gse_hour,by_gse_hour,bz_gse_hour])

In [ ]:
magdata_numpy=np.array(magdata)

In [ ]:
magdata_torch= torch.from_numpy(magdata_numpy)

In [ ]:
m3=magdata_torch.unsqueeze(-1)

In [ ]:
datalen=m3.shape[0]

In [ ]:
m3=m3.expand(datalen,3,3)

In [ ]:
m3=m3.transpose(1,0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        num_layers: int) -> None:
        super(Seq2Seq, self).__init__()

        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.dnn = nn.Sequential(
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, seq: torch.Tensor) -> torch.Tensor:
        output, hn = self.rnn(seq)
        output = self.dnn(output)
        return output

In [ ]:
model1 = Seq2Seq(3,3,3,3)
model1.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive', "dscovr_proton3.pt")))

<All keys matched successfully>

In [ ]:
proton=model1(m3.float())

In [ ]:
nprot3=[]
for i in range(proton.shape[1]):
  pnp=0
  for j in range(proton.shape[0]):
    pnp+=proton[j][i][2].detach().numpy()
  pnp=pnp/proton.shape[0]
  nprot3.append(pnp)

In [ ]:
nprot3_torch=torch.from_numpy(np.array(nprot3))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = open('/content/drive/MyDrive/dstnew.txt', "r")
lines = df.readlines()
dstarray=[]
count=0
for index, line in enumerate(lines):
      lines[index] = line.strip().split()
      del lines[index][0]
      del lines[index][0]
      l=len(lines[index])
      for i in range(l):
        count+=1
        if count!=7687:
         dstarray.append(int(lines[index][i]))

In [ ]:
dstarrayn=np.array(dstarray)

In [ ]:
dsttensor=torch.from_numpy(dstarrayn)

In [ ]:
class WindProtonDatasetToDST():
	def __init__(self):
		self.x_prime = torch.tensor([]).float()
		self.dst = torch.tensor([]).float()
		self.x_prime = torch.cat([self.x_prime, nprot3_torch.unsqueeze(-1).float()], dim=0)
		self.dst = torch.cat([self.dst, dsttensor[:self.x_prime.shape[0]].unsqueeze(-1).float()], dim=0)
		
		print("total x_prime shape:", self.x_prime.shape)
		print("total dst shape:", self.dst.shape)


	def __len__(self) -> int:
		return self.dst.shape[0]

	def __getitem__(self, index):
		return self.x_prime[index],self.dst[index]

In [ ]:
  d=WindProtonDatasetToDST()
  data_len = len(d)
  train_ratio,test_ratio = 0.7,0.3
  train_len= int(data_len*train_ratio)
  test_len = data_len - train_len 
  train_dataset,test_dataset = random_split(d, [train_len,test_len])
  print(f"train,test dataset len: {len(train_dataset)}, {len(test_dataset)}")
  train_loader = DataLoader(train_dataset, 10000, shuffle=True)
  test_loader = DataLoader(test_dataset, 10000, shuffle=False)
  model = Seq2Seq(1,1,1,3)
  print(model)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.8)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, min_lr=5e-5)
  loss_train=0
  prev_train=100
  while abs(loss_train-prev_train)>0.001:
    model.train()
    prev_train=loss_train
    loss_train, acc_train, iterations = 0, 0, 0
    for x_prime, dst in train_loader:
      outputs = model(x_prime)
      criterion = torch.nn.MSELoss()
      weight=torch.where(dst<=0,10.,20.)
      newdst=dst/weight
      loss = criterion(outputs, newdst)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      iterations+= 1
      loss_train += loss.item()
    loss_train /= iterations
    print(f"train_loss: {loss_train:.4f}")
    torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive', "proton_to_dst_new19.pt"))
		



total x_prime shape: torch.Size([8604, 1])
total dst shape: torch.Size([8604, 1])
train,test dataset len: 6022, 2582
Seq2Seq(
  (rnn): GRU(1, 1, num_layers=3, batch_first=True)
  (dnn): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
)
train_loss: 1.3886
train_loss: 3.1368
train_loss: 1.4350
train_loss: 1.5535
train_loss: 2.2321
train_loss: 1.9483
train_loss: 1.3822
train_loss: 1.2705
train_loss: 1.5849
train_loss: 1.7720
train_loss: 1.6024
train_loss: 1.3274
train_loss: 1.2470
train_loss: 1.3890
train_loss: 1.5280
train_loss: 1.4911
train_loss: 1.3388
train_loss: 1.2435
train_loss: 1.2852
train_loss: 1.3823
train_loss: 1.4047
train_loss: 1.3325
train_loss: 1.2547
train_loss: 1.2762
train_loss: 1.3038
train_loss: 1.3256
train_loss: 1.3021
train_loss: 1.2574
train_loss: 1.2418
train_loss: 1.2649
train_loss: 1.2889
train_loss: 1.2819
train_loss: 1.2546
train_loss: 1.2412
train_loss: 1.2535
train_loss: 1.2696
train_loss: 1.2665
train_loss: 1.2497
train_loss: 1.24

In [ ]:

  model = Seq2Seq(1,1,1,3)
  model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive', "proton_to_dst_new19.pt")))
  model.eval()
  with torch.no_grad():
    loss_test, acc_test,iterations = 0, 0, 0 
    for x_prime, dst in test_loader:
      outputs = model(x_prime)
      weight = torch.where(dst<=0, 10., 20.)
      newdst=dst/weight
      loss = criterion(outputs, newdst)
      iterations += 1
      loss_test += loss.item()
      for i in range(outputs.shape[0]):
       if loss_test<1.1 and abs(outputs[i]-newdst[i])<=loss_test:
         acc_test+=1
       elif abs(outputs[i]-newdst[i])<1.4:
         acc_test+=1
    loss_test /= iterations
    acc_test /= outputs.shape[0]
  print(f"test_acc: {acc_test*100:.4f},test_loss: {loss_test:.4f}")

test_acc: 87.8389,test_loss: 1.2508
